# Title
[]()

In [1]:

import sys
sys.path.append(r"/home/silvhua/custom_python")
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/')
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/app')
from silvhua import *
from chat_functions import openai_models

In [2]:
path = '/home/silvhua/repositories/notion/data/'


# List available models

In [3]:
models = openai_models()
models

gpt-4-1106-preview
gpt-3.5-turbo-0301
gpt-3.5-turbo-0125
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-16k
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-3.5-turbo
gpt-4
gpt-4-0613
gpt-4-vision-preview
gpt-3.5-turbo-0613
gpt-3.5-turbo-1106
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914


[Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-0301', created=1677649963, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-0125', created=1706048358, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-16k-0613', created=1685474247, object='model', owned_by='openai'),
 Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'),
 Model(id='gpt-4-0125-preview', created=1706037612, object='model', owned_by='system'),
 Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'),
 Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'),
 Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'),
 Model(id='gpt-4-vision-preview', created=1698894917, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-0613', created=168

# Iteration 1

# *End of Page*